Colab para extraer datos de Ventas de los registros diarios de Amarillo de Aveliza

In [1]:
import pandas as pd
import numpy as np
print("Listo")

Listo


In [ ]:
#db = pd.read_csv("C:/Users/rober/OneDrive - Baltrek/ventas_consolidadas.csv")
db = pd.read_csv("ventas_consolidadas.csv")
db.head(4)

,0,1,2,3,4,5,6,7,8,9,10,11,12,Fecha,Archivo
0,NaN,Cobrador Asignado,Código de Zona de Atención,Id Cliente,Cliente,Ave,Tipo Pedido,Peso Neto,Unidades,Desc P T.,Desc C,Neto Final,Precio,4/4/2025,PRECIO AMARILLO 04/04/25
1,POLLO,HERNANDEZ VELASCO RONNY JOSE,2,2794,ASTO AYJA ESTHER ELIZABETH,P M,PRECIO VIVO,36.25,15,0,0,36.25,7.2,4/4/2025,PRECIO AMARILLO 04/04/25
2,POLLO,CARRASCO SALINAS JOSELYN JANET,2,2773,BALBUENA MENDOZA MARIA LUZ,P M,VIVO,1858,712,0,0,1858,5.58,4/4/2025,PRECIO AMARILLO 04/04/25
3,POLLO,HERNANDEZ VELASCO RONNY JOSE,2,2728,CALATAYUD PAREJA BRYAN ALFREDO,P M,PRECIO VIVO,38.2,15,0,0,38.2,7.3,4/4/2025,PRECIO AMARILLO 04/04/25


In [3]:
db = db.rename(
    columns = {"0":"Ave1",
               "1":"Vendedor",
               "2":"Código de Zona de Atención",
               "3":"Id Cliente",
               "4":"Cliente",
               "5":"Ave",
               "6":"Tipo Pedido",
               "7":"Peso Neto",
               "8":"Unidades",
               "9":"Desc P T.",
               "10":"Desc C",
               "11":"Neto Final",
               "12":"Precio"}
)

In [4]:
#ELimina fila 1 y resetea índice
db = db.drop(index=0).reset_index(drop=True)

In [5]:
db.head(4)

,Ave1,Vendedor,Código de Zona de Atención,Id Cliente,Cliente,Ave,Tipo Pedido,Peso Neto,Unidades,Desc P T.,Desc C,Neto Final,Precio,Fecha,Archivo
0,POLLO,HERNANDEZ VELASCO RONNY JOSE,2,2794,ASTO AYJA ESTHER ELIZABETH,P M,PRECIO VIVO,36.25,15,0,0,36.25,7.2,4/4/2025,PRECIO AMARILLO 04/04/25
1,POLLO,CARRASCO SALINAS JOSELYN JANET,2,2773,BALBUENA MENDOZA MARIA LUZ,P M,VIVO,1858,712,0,0,1858,5.58,4/4/2025,PRECIO AMARILLO 04/04/25
2,POLLO,HERNANDEZ VELASCO RONNY JOSE,2,2728,CALATAYUD PAREJA BRYAN ALFREDO,P M,PRECIO VIVO,38.2,15,0,0,38.2,7.3,4/4/2025,PRECIO AMARILLO 04/04/25
3,POLLO,HERNANDEZ VELASCO RONNY JOSE,2,2734,CAQUIAMARCA CACERES CARITO MILAGROS,P M,PRECIO VIVO,11.8,5,0,0,11.8,7.3,4/4/2025,PRECIO AMARILLO 04/04/25


In [ ]:
db.groupby("Vendedor")["Vendedor"].count()

In [ ]:
print(db.dtypes)

In [6]:
db["Peso Neto"] = pd.to_numeric(db["Peso Neto"], errors='coerce')

In [7]:
# Conversión segura de fechas: interpreta primero el día
db["Fecha"] = pd.to_datetime(db["Fecha"], dayfirst=True, errors='coerce')


In [8]:
# Definir las fechas en datetime
fecha_errada = pd.to_datetime("06/12/2025", dayfirst=True)  # Interpretado como 6 de diciembre
fecha_correcta = pd.to_datetime("06/01/2025", dayfirst=True)  # 6 de enero

# Reemplazar la fecha errada
db.loc[db["Fecha"] == fecha_errada, "Fecha"] = fecha_correcta


In [ ]:
dbPMPESONETO=db[db["Ave"]=="P M"].groupby("Fecha")["Peso Neto"].sum()

In [ ]:
dbPMPESONETO.head(4)

In [ ]:
#!pip install vegafusion
#!pip install vegafusion-jupyter

In [ ]:
import plotly.express as px

# Si tu resultado es una Serie (Fecha como índice), primero conviértelo a DataFrame
df_plot = dbPMPESONETO.reset_index()

fig = px.line(df_plot, x="Fecha", y="Peso Neto", title="Peso Neto diario de 'P M'")
fig.update_layout(xaxis_title="Fecha", yaxis_title="Peso Neto")
fig.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Asegúrate de que la columna "Fecha" es de tipo datetime
db["Fecha"] = pd.to_datetime(db["Fecha"])

# Filtrar fechas posteriores al 1 de enero de 2025
db_filtrado = db[db["Fecha"] > pd.to_datetime("2025-01-01")]

# Agrupar y pivotear para que cada columna sea una categoría de "Ave"
df_grouped = db_filtrado.groupby(["Fecha", "Ave"])["Peso Neto"].sum().unstack()

# Crear la gráfica
df_grouped.plot(kind='line', figsize=(12, 6), title="Peso Neto por tipo de Ave desde 01/01/2025")

plt.xlabel("Fecha")
plt.ylabel("Peso Neto")
plt.legend(title="Tipo de Ave")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import plotly.express as px
import pandas as pd

# Asegúrate de que la columna "Fecha" es de tipo datetime
db["Fecha"] = pd.to_datetime(db["Fecha"])

# Filtrar fechas posteriores al 1 de enero de 2025
db_filtrado = db[db["Fecha"] > pd.to_datetime("2025-01-01")]

# Agrupar por Fecha y Ave
df_grouped = db_filtrado.groupby(["Fecha", "Ave"])["Peso Neto"].sum().reset_index()

# Crear el gráfico de líneas
fig = px.line(
    df_grouped,
    x="Fecha",
    y="Peso Neto",
    color="Ave",
    title="Peso Neto por tipo de Ave desde el 01/01/2025"
)

fig.update_layout(
    xaxis_title="Fecha",
    yaxis_title="Peso Neto",
    legend_title="Tipo de Ave",
    template="plotly_white"
)

fig.show()


In [ ]:
#Código con Plotly (con media móvil + picos)
#import pandas as pd
import plotly.graph_objects as go

# Asegurar formato datetime
db["Fecha"] = pd.to_datetime(db["Fecha"])

# Filtrar fechas > 01/01/2025
db_filtrado = db[db["Fecha"] > pd.to_datetime("2025-01-01")]

# Agrupar por Fecha y Ave
df_grouped = db_filtrado.groupby(["Fecha", "Ave"])["Peso Neto"].sum().reset_index()

# Crear lista de figuras para graficar múltiples líneas
fig = go.Figure()

# Definir ventana de media móvil
ventana = 7  # días

# Iterar por cada tipo de ave
for ave in df_grouped["Ave"].unique():
    df_ave = df_grouped[df_grouped["Ave"] == ave].sort_values("Fecha")

    # Línea original
    fig.add_trace(go.Scatter(
        x=df_ave["Fecha"],
        y=df_ave["Peso Neto"],
        mode="lines",
        name=f"{ave} - Peso Neto",
        line=dict(width=1)
    ))

    # Media móvil
    df_ave["Media Móvil"] = df_ave["Peso Neto"].rolling(window=ventana, min_periods=1).mean()
    fig.add_trace(go.Scatter(
        x=df_ave["Fecha"],
        y=df_ave["Media Móvil"],
        mode="lines",
        name=f"{ave} - Media {ventana} días",
        line=dict(dash="dot")
    ))

    # Pico (día con mayor peso neto)
    pico = df_ave[df_ave["Peso Neto"] == df_ave["Peso Neto"].max()]
    fig.add_trace(go.Scatter(
        x=pico["Fecha"],
        y=pico["Peso Neto"],
        mode="markers+text",
        name=f"{ave} - Pico",
        text=["Pico"],
        textposition="top center",
        marker=dict(size=10, color="red", symbol="star")
    ))

# Personalizar layout
fig.update_layout(
    title="Peso Neto por tipo de Ave desde 01/01/2025 con media móvil y picos",
    xaxis_title="Fecha",
    yaxis_title="Peso Neto",
    legend_title="Tipo de Ave",
    template="plotly_white",
    hovermode="x unified"
)

fig.show()


In [ ]:
db.head(5)

In [9]:
#Versión que genera un nuevo DataFrame con las categorizaciones
import pandas as pd
import numpy as np

# Aseguramos que la columna Fecha esté en formato datetime
#db = df_original.copy()
db["Fecha"] = pd.to_datetime(db["Fecha"])

# Creamos el rango de fechas continuo
all_dates = pd.date_range(start=db["Fecha"].min(), end=db["Fecha"].max(), freq='D')

# Creamos una tabla de presencia: 1 si hay compra ese día, 0 si no
db_presence = db.groupby(["Cliente", "Fecha"]).size().unstack(fill_value=0)
db_presence = db_presence.reindex(columns=all_dates, fill_value=0).sort_index(axis=1)

# Creamos el DataFrame donde pondremos los estados: N, A, I, R
db_status = pd.DataFrame(index=db_presence.index, columns=db_presence.columns, dtype=str)

# Clasificamos el comportamiento de cada cliente
for cliente in db_presence.index:
    compras_cliente = db_presence.loc[cliente]
    estado_anterior = None
    dias_inactivo = 0
    ha_comprado_antes = False

    for fecha in compras_cliente.index:
        hay_compra = compras_cliente[fecha] > 0

        if hay_compra:
            if not ha_comprado_antes:
                estado = 'N'  # Nuevo cliente
                ha_comprado_antes = True
            elif estado_anterior == 'I':
                estado = 'R'  # Recuperado
            else:
                estado = 'A'  # Activo
            dias_inactivo = 0  # Reinicia contador de inactividad
        else:
            if not ha_comprado_antes:
                estado = np.nan  # Aún no ha sido cliente
            elif dias_inactivo >= 3:  # El cuarto día sin comprar
                estado = 'I'
            else:
                estado = estado_anterior  # Mantiene el último estado válido
            dias_inactivo += 1

        db_status.loc[cliente, fecha] = estado
        estado_anterior = estado

# Opcional: reemplazar NaN por vacío para visualización
db_status.fillna("", inplace=True)

# Mostrar resultado
db_status.head()


,2024-07-05,2024-07-06,2024-07-07,2024-07-08,2024-07-09,2024-07-10,2024-07-11,2024-07-12,2024-07-13,2024-07-14,...,2025-03-26,2025-03-27,2025-03-28,2025-03-29,2025-03-30,2025-03-31,2025-04-01,2025-04-02,2025-04-03,2025-04-04
Cliente,,,,,,,,,,,,,,,,,,,,,
ABAD AIRA CLIVER MARADONA,,,,,,,,,,,...,I,I,I,I,I,I,I,I,I,I
ABAD AIRA YUNIOR ADOLFO,,,,,,,,,,,...,I,I,I,I,I,I,I,I,I,I
ABANTO CAROL,,,,,,,,,,,...,R,R,R,A,A,A,A,A,A,A
ABANTO GUEVARA ARTURO,,,,,,,,,,,...,I,I,I,I,I,I,I,I,I,I
ACARO JULCA MERLY JULISSA,,,,,,,,,,,...,I,I,I,I,I,I,I,I,I,I


In [12]:
import pandas as pd
import numpy as np

# Copiamos y preprocesamos el DataFrame original

db["Fecha"] = pd.to_datetime(db["Fecha"])

# Fecha de inicio para la clasificación
fecha_inicio = pd.to_datetime("2025-01-01")

# Creamos el rango completo de fechas, desde la fecha mínima hasta la máxima,
# pero filtraremos para mostrar solo desde `fecha_inicio`
all_dates = pd.date_range(start=db["Fecha"].min(), end=db["Fecha"].max(), freq='D')

# Tabla de presencia: 1 si hay compra ese día, 0 si no
db_presence = db.groupby(["Cliente", "Fecha"]).size().unstack(fill_value=0)
db_presence = db_presence.reindex(columns=all_dates, fill_value=0).sort_index(axis=1)

# DataFrame de clasificación de estados
db_status = pd.DataFrame(index=db_presence.index, columns=db_presence.columns, dtype=str)

# Clasificación por cliente
for cliente in db_presence.index:
    compras_cliente = db_presence.loc[cliente]
    estado_anterior = None
    dias_inactivo = 0
    ha_comprado_antes = False

    for fecha in compras_cliente.index:
        hay_compra = compras_cliente[fecha] > 0

        if hay_compra:
            if not ha_comprado_antes:
                estado = 'N'
                ha_comprado_antes = True
            elif estado_anterior == 'I':
                estado = 'R'
            else:
                estado = 'A'
            dias_inactivo = 0
        else:
            if not ha_comprado_antes:
                estado = np.nan
            elif dias_inactivo >= 3:
                estado = 'I'
            else:
                estado = estado_anterior
            dias_inactivo += 1

        db_status.loc[cliente, fecha] = estado
        estado_anterior = estado

# Limpiar y filtrar solo desde fecha_inicio
db_status_filtered = db_status.loc[:, db_status.columns >= fecha_inicio]

# Reiniciar el índice para que 'Clientes' sea una columna (para exportar correctamente)
db_status_filtered.reset_index(inplace=True)
db_status_filtered.rename(columns={'index': 'Clientes'}, inplace=True)

# Exportar a CSV
db_status_filtered.to_csv("clasificacion_clientes.csv", index=False, encoding='utf-8-sig')

# Mostrar parte del resultado
db_status_filtered.head()


C:\Users\rober\AppData\Local\Temp\ipykernel_13048\3800097902.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_status_filtered.rename(columns={'index': 'Clientes'}, inplace=True)


,Cliente,2025-01-01 00:00:00,2025-01-02 00:00:00,2025-01-03 00:00:00,2025-01-04 00:00:00,2025-01-05 00:00:00,2025-01-06 00:00:00,2025-01-07 00:00:00,2025-01-08 00:00:00,2025-01-09 00:00:00,...,2025-03-26 00:00:00,2025-03-27 00:00:00,2025-03-28 00:00:00,2025-03-29 00:00:00,2025-03-30 00:00:00,2025-03-31 00:00:00,2025-04-01 00:00:00,2025-04-02 00:00:00,2025-04-03 00:00:00,2025-04-04 00:00:00
0,ABAD AIRA CLIVER MARADONA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,I,I,I,I,I,I,I,I,I,I
1,ABAD AIRA YUNIOR ADOLFO,NaN,NaN,NaN,NaN,NaN,N,N,N,N,...,I,I,I,I,I,I,I,I,I,I
2,ABANTO CAROL,NaN,NaN,NaN,NaN,NaN,NaN,N,A,A,...,R,R,R,A,A,A,A,A,A,A
3,ABANTO GUEVARA ARTURO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,I,I,I,I,I,I,I,I,I,I
4,ACARO JULCA MERLY JULISSA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,I,I,I,I,I,I,I,I,I,I


In [10]:
ruta_salida = "C:/Users/rober/OneDrive - Baltrek/Archivos de Cesar Meza - 09 AVELIZA/RS/BALTREK/Clasificados.csv"
db_status.to_csv(ruta_salida, index=False)

print(f"Archivo guardado en: {ruta_salida}")

Archivo guardado en: C:/Users/rober/OneDrive - Baltrek/Archivos de Cesar Meza - 09 AVELIZA/RS/BALTREK/Clasificados.csv


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

# Nos aseguramos de que las fechas están en columnas tipo datetime
db_status_filtered.columns = ['Cliente'] + list(pd.to_datetime(db_status_filtered.columns[1:]))

# Unimos la información del vendedor (asumimos que "db" tiene columnas: Vendedor, Cliente, Fecha)
clientes_vendedor = db[['Cliente', 'Vendedor']].drop_duplicates()
df_vendedores = db_status_filtered.merge(clientes_vendedor, on='Cliente', how='left')

# Función para preparar y graficar por vendedor
def graficar_por_vendedor(vendedor_seleccionado):
    df_filtrado = df_vendedores[df_vendedores['Vendedor'] == vendedor_seleccionado]
    
    # Reindexar por cliente y pivotear
    df_melt = df_filtrado.melt(id_vars=['Cliente', 'Vendedor'], var_name='Fecha', value_name='Estado')
    
    # Contar estados por fecha
    conteo = df_melt.groupby(['Fecha', 'Estado']).size().unstack(fill_value=0)
    
    # Asegurar que estén todas las columnas necesarias
    for col in ['N', 'R', 'I']:
        if col not in conteo.columns:
            conteo[col] = 0
    
    # Calcular la métrica neta
    conteo['Neto'] = conteo['N'] + conteo['R'] - conteo['I']
    
    # Crear el gráfico
    plt.figure(figsize=(16, 8))
    
    # Área para inactivos
    plt.fill_between(conteo.index, conteo['I'], label='Inactivos', color='lightgray', alpha=0.5)
    
    # Barras apiladas para Nuevos y Recuperados
    plt.bar(conteo.index, conteo['N'], label='Nuevos', color='skyblue')
    plt.bar(conteo.index, conteo['R'], bottom=conteo['N'], label='Recuperados', color='green')
    
    # Línea para Neto
    plt.plot(conteo.index, conteo['Neto'], label='Neto (N + R - I)', color='red', linewidth=2)
    
    # Estética
    plt.title(f"Clasificación de clientes - Vendedor: {vendedor_seleccionado}")
    plt.xlabel("Fecha")
    plt.ylabel("Cantidad de Clientes")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Widget para seleccionar vendedor
vendedores_unicos = sorted(db['Vendedor'].dropna().unique())
selector_vendedor = widgets.Dropdown(
    options=vendedores_unicos,
    description='Vendedor:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Vincular selector con función
widgets.interact(graficar_por_vendedor, vendedor_seleccionado=selector_vendedor)


interactive(children=(Dropdown(description='Vendedor:', layout=Layout(width='50%'), options=('-', 'ABAD AIRA C…

<function __main__.graficar_por_vendedor(vendedor_seleccionado)>

In [16]:
#Agregamos la columna "Monto" como el producto de "Neto Final" × "Precio":
# Asegurar tipos numéricos
db["Neto Final"] = pd.to_numeric(db["Neto Final"], errors='coerce')
db["Precio"] = pd.to_numeric(db["Precio"], errors='coerce')

# Calcular el monto total por pedido
db["Monto"] = db["Neto Final"] * db["Precio"]

# Verificamos
db[["Fecha", "Cliente", "Vendedor", "Tipo Pedido", "Neto Final", "Precio", "Monto"]].head()


,Fecha,Cliente,Vendedor,Tipo Pedido,Neto Final,Precio,Monto
0,2025-04-04,ASTO AYJA ESTHER ELIZABETH,HERNANDEZ VELASCO RONNY JOSE,PRECIO VIVO,36.25,7.20,261.000
1,2025-04-04,BALBUENA MENDOZA MARIA LUZ,CARRASCO SALINAS JOSELYN JANET,VIVO,1858.00,5.58,10367.640
2,2025-04-04,CALATAYUD PAREJA BRYAN ALFREDO,HERNANDEZ VELASCO RONNY JOSE,PRECIO VIVO,38.20,7.30,278.860
3,2025-04-04,CAQUIAMARCA CACERES CARITO MILAGROS,HERNANDEZ VELASCO RONNY JOSE,PRECIO VIVO,11.80,7.30,86.140
4,2025-04-04,CAYHUALLA CHOQUE ALCIDES,CARRASCO SALINAS JOSELYN JANET,VIVO,935.20,5.58,5218.416


In [19]:
#Gráfica de líneas de "Monto" y "Neto Final", filtrable por Cliente, Vendedor, y Tipo Pedido
#Usaremos ipywidgets para los filtros y matplotlib para el gráfico:

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Asegurar formato de fechas
db["Fecha"] = pd.to_datetime(db["Fecha"])

# Añadir opción "Todos" a cada filtro
clientes = ["Todos"] + sorted(db["Cliente"].dropna().unique())
vendedores = ["Todos"] + sorted(db["Vendedor"].dropna().unique())
tipos_pedido = ["Todos"] + sorted(db["Tipo Pedido"].dropna().unique())

# Widgets
cliente_widget = widgets.Dropdown(
    options=clientes,
    value="Todos",
    description="Cliente:",
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

vendedor_widget = widgets.Dropdown(
    options=vendedores,
    value="Todos",
    description="Vendedor:",
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

tipo_pedido_widget = widgets.Dropdown(
    options=tipos_pedido,
    value="Todos",
    description="Tipo de Pedido:",
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

# Función para graficar aplicando filtros opcionales
def graficar_monto_neto(cliente, vendedor, tipo_pedido):
    df_filtro = db.copy()

    if cliente != "Todos":
        df_filtro = df_filtro[df_filtro["Cliente"] == cliente]
    if vendedor != "Todos":
        df_filtro = df_filtro[df_filtro["Vendedor"] == vendedor]
    if tipo_pedido != "Todos":
        df_filtro = df_filtro[df_filtro["Tipo Pedido"] == tipo_pedido]

    if df_filtro.empty:
        print("No hay datos con los filtros seleccionados.")
        return

    # Agrupación diaria
    df_diario = df_filtro.groupby("Fecha")[["Monto", "Neto Final"]].sum().sort_index()

    # Gráfico
    plt.figure(figsize=(14, 6))
    plt.plot(df_diario.index, df_diario["Monto"], label="Monto (S/.)", color="blue", linewidth=2)
    plt.plot(df_diario.index, df_diario["Neto Final"], label="Neto Final", color="orange", linewidth=2)

    titulo = "Evolución de Monto y Neto Final"
    subtitulo = f"Cliente: {cliente} | Vendedor: {vendedor} | Tipo: {tipo_pedido}"
    plt.title(f"{titulo}\n{subtitulo}")
    plt.xlabel("Fecha")
    plt.ylabel("Valor")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Interfaz interactiva
widgets.interact(
    graficar_monto_neto,
    cliente=cliente_widget,
    vendedor=vendedor_widget,
    tipo_pedido=tipo_pedido_widget
)


interactive(children=(Dropdown(description='Cliente:', layout=Layout(width='50%'), options=('Todos', 'ABAD AIR…

<function __main__.graficar_monto_neto(cliente, vendedor, tipo_pedido)>